In [1]:
!wget -O dataset.zip 'https://lfs.aminer.cn/lab-datasets/citation/acm.v9.zip'


--2025-01-16 19:56:44--  https://lfs.aminer.cn/lab-datasets/citation/acm.v9.zip
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 686542584 (655M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 654.74M  10.3MB/s    in 67s     

2025-01-16 19:57:52 (9.81 MB/s) - ‘dataset.zip’ saved [686542584/686542584]



In [2]:
!unzip dataset.zip -d /content/dataset


Archive:  dataset.zip
  inflating: /content/dataset/acm.txt  


In [10]:
import pandas as pd

# Path to the dataset file
file_path = '/content/dataset/acm.txt'

# Lists to store parsed data
titles = []
authors = []
years = []
venues = []
indices = []

# Open and parse the file line by line
with open(file_path, 'r') as file:
    record = {}  # Temporary dictionary to store each record
    for line in file:
        line = line.strip()
        if line.startswith("#*"):
            record['title'] = line[2:]  # Title
        elif line.startswith("#@"):
            record['authors'] = line[2:]  # Authors
        elif line.startswith("#t"):
            record['year'] = line[2:]  # Year
        elif line.startswith("#c"):
            record['venue'] = line[2:]  # Venue
        elif line.startswith("#index"):
            record['index'] = line[6:]  # Index
        elif line == "":  # Empty line indicates end of a record
            if record:  # If the record dictionary is not empty
                titles.append(record.get('title', ''))
                authors.append(record.get('authors', ''))
                years.append(record.get('year', ''))
                venues.append(record.get('venue', ''))
                indices.append(record.get('index', ''))
                record = {}  # Reset for the next record

# Create a DataFrame from the lists
df = pd.DataFrame({
    'title': titles,
    'authors': authors,
    'year': years,
    'venue': venues,
    'index': indices
})

# Preview the first few rows
print(df.head())


                                               title  \
0  MOSFET table look-up models for circuit simula...   
1  The verification of the protection mechanisms ...   
2  Another view of functional and multivalued dep...   
3     Entity-relationship diagrams which are in BCNF   
4                          The computer comes of age   

                                             authors  year  \
0                                                     1984   
1                                   Virgil D. Gligor  1984   
2                           M. Gyssens, J. Paredaens  1984   
3  Sushil Jajodia, Peter A. Ng, Frederick N. Spri...  1984   
4                                        Rene Moreau  1984   

                                           venue index  
0                  Integration, the VLSI Journal     1  
1  International Journal of Parallel Programming     2  
2  International Journal of Parallel Programming     3  
3  International Journal of Parallel Programming     4  
4    

In [11]:
import matplotlib.pyplot as plt
from collections import defaultdict

In [13]:
# Number of distinct authors
authors_set = set()
df['authors'].str.split(',').dropna().apply(authors_set.update)  # Assuming authors are comma-separated
num_authors = len(authors_set)
print(f"Number of distinct authors: {num_authors}")

# Number of distinct venues
num_venues = df['venue'].nunique()
print(f"Number of distinct venues: {num_venues}")

# Number of distinct publications (based on unique titles)
num_publications = df['title'].nunique()
print(f"Number of distinct publications: {num_publications}")

# Number of distinct references/citations
# Note: Your dataset example doesn't include references field. Remove this if not applicable.
if 'references' in df.columns:
    references_set = set()
    df['references'].dropna().str.split(';').apply(references_set.update)  # Assuming references are semicolon-separated
    num_references = len(references_set)
    print(f"Number of distinct citations/references: {num_references}")
else:
    print("References field not found in dataset.")


Number of distinct authors: 2027009
Number of distinct venues: 273330
Number of distinct publications: 2183552
References field not found in dataset.
